[this doc on github](https://github.com/dotnet/interactive/tree/main/samples/notebooks/fsharp/Samples)

This demonstrates the use of `Microsoft.Data.Analysis` data frames with F#.You can open this example online using [MyBinder](https://mybinder.org/v2/gh/dotnet/interactive/main?filepath=fsharp%2FSamples%2FDataFrame-Getting%20Started.ipynb).

### Referencing the package


First, get the package and open the namespaces:

In [ ]:
#r "nuget:Microsoft.Data.Analysis,0.2.0"
#r "nuget: XPlot.Plotly.Interactive, 4.0.2"

open Microsoft.Data.Analysis

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.Data.Analysis version 0.2.0

Installed package XPlot.Plotly.Interactive version 4.0.2

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

### Creating a data frame

Create 3 columns to hold values of types `DateTime`, `int`, and `string`

In [ ]:
let dateTimes = PrimitiveDataFrameColumn<DateTime>("DateTimes") // Default length is 0.
let ints = PrimitiveDataFrameColumn<int>("Ints", 3L) // Makes a column of Length 3. Filles with nulls initially.
let strings = StringDataFrameColumn("Strings", 3L)

Add some datetimes

In [ ]:
dateTimes.Append(DateTime.Parse("2019/01/01"))
dateTimes.Append(DateTime.Parse("2019/01/01"))
dateTimes.Append(DateTime.Parse("2019/01/02"))

Create a `DataFrame` with 3 columns

In [ ]:
let df = DataFrame([dateTimes; ints; strings]: DataFrameColumn list)

### Adding better default formatting for data frames

Create a formatter for data frames and data frame rows.

In [ ]:
module DateFrameFormatter = 
    
    // Locally open the F# HTML DSL.
    open Html

    let maxRows = 20

    Formatter.Register<DataFrame>((fun (df: DataFrame) (writer: TextWriter) ->

        let take = 20
        table [] [
          thead [] [
            th [] [ str "Index" ]
            for c in df.Columns do
              th [] [ str c.Name]
          ]
          tbody [] [
            for i in 0 .. min maxRows (int df.Rows.Count - 1) do
              tr [] [
                td [] [ i ]
                for o in df.Rows.[int64 i] do
                  td [] [ o ]
              ]
          ]
        ]
        |> writer.Write

    ), mimeType = "text/html")
    
    Formatter.Register<DataFrameRow>((fun (row: DataFrameRow) (writer: TextWriter) ->

        table [] [
          tbody [] [
            tr [] [
              for o in row do
                td [] [ o ] 
            ]
          ]
        ]
        |> writer.Write

    ), mimeType = "text/html")
    

Now view the data frame: 

In [ ]:
df

0,2019-01-01 00:00:00Z,<null>,<null>
1,2019-01-01 00:00:00Z,<null>,<null>
2,2019-01-02 00:00:00Z,<null>,<null>


### Modifying data frames

Change a value directly through df:

In [ ]:
df.[0L, 1] <- 10
df

0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,100,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


We can also modify the values in the columns through indexers defined in `PrimitiveDataColumn` and `StringColumn`

In [ ]:
ints.[1L] <- Nullable 100
strings.[1L] <- "Foo!"
df

0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,100,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


Check the data type

In [ ]:
df.Info()

0,DataType,System.DateTime,System.Int32,System.String
1,Length (excluding null values),3,2,3


The `DataFrame` and the base `DataFrameColumn` class that all columns derive from expose a number of useful APIs: binary operations, computations, joins, merges, handling missing values and more.

In [ ]:
df.["Ints"].Add(5, inPlace=true)
df

0,2019-01-01 00:00:00Z,15,<null>
1,2019-01-01 00:00:00Z,105,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


In [ ]:
df.["Ints"] <- (ints / 5) * 100
df

0,2019-01-01 00:00:00Z,300,<null>
1,2019-01-01 00:00:00Z,2100,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


Let's `null` it up!

In [ ]:
df.["Ints"].FillNulls(-1, inPlace=true)
df.["Strings"].FillNulls("Bar", inPlace=true)
df

0,2019-01-01 00:00:00Z,300,Bar
1,2019-01-01 00:00:00Z,2100,Foo!
2,2019-01-02 00:00:00Z,-1,Bar


DataFrame exposes `Columns` property that we can enumerate over to access our columns. Here's how you can access the first row, though.

In [ ]:
let row0 = df.Rows.[0L]
row0

2019-01-01 00:00:00Z,300,Bar


In [ ]:
row0

2019-01-01 00:00:00Z,300,Bar


### Filtering and sorting data frames

Let's take a look at `Filter`, `Sort`, and `GroupBy`.

In [ ]:
// Sort our dataframe using the Ints column
df.Sort("Ints", ascending=true)

0,2019-01-02 00:00:00Z,-1,Bar
1,2019-01-01 00:00:00Z,300,Bar
2,2019-01-01 00:00:00Z,2100,Foo!


In [ ]:
// GroupBy
let grouped = df.GroupBy("DateTimes")
// Count of values in each group
grouped.Count()

0,2019-01-01 00:00:00Z,2,2
1,2019-01-02 00:00:00Z,1,1


In [ ]:
let intGroupSum = grouped.Sum("Ints");
intGroupSum

0,2019-01-01 00:00:00Z,2400
1,2019-01-02 00:00:00Z,-1


### Charting columns from data frames

In [ ]:
open XPlot.Plotly
open System.Linq

In [ ]:
#r "nuget:MathNet.Numerics"

Installed package MathNet.Numerics version 4.15.0

In [ ]:
open MathNet.Numerics.Distributions

In [ ]:
let mean = 0.0
let stdDev = 0.1

let normalDist = new Normal(mean, stdDev);

In [ ]:
let doubles = PrimitiveDataFrameColumn<double>("Normal Distribution", normalDist.Samples().Take(1000));
// let ints = PrimitiveDataFrameColumn<int>("Ints", 3L) 
display(Chart.Plot(Histogram(x = doubles, nbinsx = 30)));

<!DOCTYPE html>